In [ ]:
import requests
import os
import time
from datetime import datetime
import gc

# Diretório para salvar os arquivos
output_dir = 'dados_anp_ca'
os.makedirs(output_dir, exist_ok=True)

# Base URL
base_url = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-'

# Anos de 2004 a 2025
anos = range(2004, 2026)  # Até 2025 inclusive

# Semestres: apenas 01 (1º semestre) e 02 (2º semestre)
semestres = ['01', '02']

def download_file(url, filepath, chunk_size=8192, max_retries=5, timeout=60):
    """
    Baixa o arquivo em chunks com retry em caso de falha para economizar memória e persistir no download.
    """
    for attempt in range(max_retries):
        try:
            # Remove arquivo parcial se existir de tentativas anteriores
            if os.path.exists(filepath):
                os.remove(filepath)
            
            print(f"Tentativa {attempt + 1}/{max_retries} para {url}")
            
            response = requests.get(url, stream=True, timeout=timeout)
            response.raise_for_status()  # Levanta exceção para erros HTTP
            
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        downloaded_size += len(chunk)
                        # Opcional: progresso se quiser, mas mantém simples
                        if total_size > 0:
                            progress = (downloaded_size / total_size) * 100
                            print(f"\rProgresso: {progress:.1f}%", end='', flush=True)
            
            # Verifica se o arquivo foi baixado completamente (tamanho > 0)
            if os.path.getsize(filepath) > 0:
                print(f"\nArquivo baixado com sucesso: {filepath} (tamanho: {os.path.getsize(filepath)} bytes)")
                return True
            else:
                print(f"\nArquivo vazio após download: {filepath}. Tentando novamente...")
                os.remove(filepath)
                continue
                
        except requests.exceptions.RequestException as e:
            print(f"\nTentativa {attempt + 1} falhou: {e}")
            if os.path.exists(filepath):
                os.remove(filepath)
            if attempt < max_retries - 1:
                wait_time = (2 ** attempt) + 1  # Backoff exponencial: 3s, 5s, 9s, etc.
                print(f"Aguardando {wait_time}s antes da próxima tentativa...")
                time.sleep(wait_time)
            else:
                print(f"Falha após {max_retries} tentativas para {url}")
                return False
        except Exception as e:
            print(f"\nErro inesperado na tentativa {attempt + 1}: {e}")
            if os.path.exists(filepath):
                os.remove(filepath)
            if attempt < max_retries - 1:
                time.sleep(2)
            else:
                return False
    
    return False

# Loop principal
sucessos = 0
falhas = 0

for ano in anos:
    for semestre in semestres:
        url = f"{base_url}{ano}-{semestre}.csv"
        filename = f"ca-{ano}-{semestre}.csv"
        filepath = os.path.join(output_dir, filename)
        
        if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
            print(f"Arquivo já existe e é válido: {filepath}. Pulando.")
            sucessos += 1
            continue
        
        success = download_file(url, filepath)
        
        if success:
            sucessos += 1
            # Limpa memória após cada download
            gc.collect()
        else:
            falhas += 1
            print(f"Falha final em {url}. Arquivo não foi baixado.")

print(f"\nDownload concluído! Sucessos: {sucessos}, Falhas: {falhas}")

Tentativa 1/5 para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2004-01.csv
Progresso: 0.5%

Progresso: 100.0%
Arquivo baixado com sucesso: dados_anp_ca\ca-2004-01.csv (tamanho: 48637859 bytes)
Tentativa 1/5 para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2004-02.csv
Progresso: 100.0%
Arquivo baixado com sucesso: dados_anp_ca\ca-2004-02.csv (tamanho: 158110155 bytes)
Tentativa 1/5 para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2005-01.csv
Progresso: 100.0%
Arquivo baixado com sucesso: dados_anp_ca\ca-2005-01.csv (tamanho: 157737721 bytes)
Tentativa 1/5 para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2005-02.csv
Progresso: 100.0%
Arquivo baixado com sucesso: dados_anp_ca\ca-2005-02.csv (tamanho: 127779846 bytes)
Tentativa 1/5 para https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2006-01.csv
Progresso: 100.0%
Arquivo baixado com sucesso: dados_anp_ca\ca-2006-01.csv (tamanho: 150271698 bytes)
Tentativa 1